In [1]:
import sys
sys.path.insert(1, '/home/elinfi/MasterCode/clustering/class/')

In [2]:
import random
import kmedoids

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from time import time
from k_medoids import KMedoids
from mpl_toolkits.mplot3d import Axes3D
from dissimilarity_matrix import DissimilarityMatrix
from data_preparation import DataPreparation
from sklearn_extra.cluster import KMedoids



Create examples of usage of dissimilarity matrices.

In [3]:
# get path to multi resolution hic data
path_wt = '/home/elinfi/coolers/HiC_wt_001.mcool'
path_cancer = '/home/elinfi/coolers/HiC_cancer_001.mcool'

# resolution
resolution = 16000

# region of genome
region = 'chr4:10M-15M'

In [4]:
# create objects of class
wt = DataPreparation(path_wt, resolution, region)
cancer = DataPreparation(path_cancer, resolution, region)

In [5]:
# divide matrices
diff = wt.divide(cancer)

/home/elinfi/MasterCode/clustering/class/data_preparation.py:67: RuntimeWarning: divide by zero encountered in true_divide
  diff = self.matrix/other.matrix
/home/elinfi/MasterCode/clustering/class/data_preparation.py:67: RuntimeWarning: invalid value encountered in true_divide
  diff = self.matrix/other.matrix


In [6]:
# calculate the distance matrix
dissimilarity = DissimilarityMatrix(diff)

# using scipy.pdist
start = time()
distmat_scipy = dissimilarity.scipy_dist('euclidean', 0, 2)
end = time()
print(f"Scipy: {end - start}")

#using sklearn.pairwise_distance
start = time()
distmat_sklearn = dissimilarity.sklearn_dist('euclidean')
end = time()
print(f"Sklearn: {end - start}")

Scipy: 6.778463840484619


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# check if the two distance matrices are equal
print(np.sum(distmat_scipy != distmat_sklearn))

In [ ]:
# find clusters using kmedoids fasterpam
c_scipy = kmedoids.fasterpam(distmat_scipy, 20, random_state=19)
c_sklearn = kmedoids.fasterpam(distmat_sklearn, 20, random_state=19)

In [ ]:
# get labels for all clusters
labels_scipy = c_scipy.labels
labels_sklearn = c_sklearn.labels

In [ ]:
# kmedoids_sklearn = KMedoids(n_clusters=20, random_state=0).fit(X)

In [ ]:
# transform cluster labels to 2D array
n = dissimilarity.n
clusters_scipy = np.zeros((n, n))
upper_triag_idx = np.triu_indices(n)
clusters_scipy[upper_triag_idx] = labels_scipy
clusters_scipy = clusters_scipy + clusters_scipy.T
clusters_scipy[np.diag_indices(n)] /= 2

In [ ]:
# transform cluster labels to 2D array
n = dissimilarity.n
clusters_sklearn = np.zeros((n, n))
upper_triag_idx = np.triu_indices(n)
clusters_sklearn[upper_triag_idx] = labels_sklearn
clusters_sklearn = clusters_sklearn + clusters_sklearn.T
clusters_sklearn[np.diag_indices(n)] /= 2

In [ ]:
# create discrete colormap
cmap = plt.cm.gnuplot2
bounds = np.linspace(0, 20, 21)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
f, axs = plt.subplots(figsize=(20, 40),
                      nrows=1,
                      ncols=2,
                      sharex=True, sharey=False)

ax = axs[0]
#im = ax.imshow(diff,
#               norm=mpl.colors.LogNorm(),
#               cmap='bwr')
im = ax.imshow(clusters_scipy,
              norm=norm,
              cmap=cmap,
              alpha=1)
plt.colorbar(im, fraction=0.046, pad=0.04, label='counts (log)', ax=ax)

ax = axs[1]
im = ax.imshow(clusters_sklearn,
              norm=norm,
              cmap=cmap,
              alpha=1)
plt.colorbar(im, fraction=0.046, pad=0.04, label='counts (log)', ax=ax)

"""
ax = axs[1]
im = ax.imshow(diff,
               norm=mpl.colors.LogNorm(),
               cmap='bwr')
plt.colorbar(im, fraction=0.046, pad=0.04, label='counts (log)', ax=ax)
ax.set(title='Wild type raw data')
"""